In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
import numpy as np

# Анализ тональности отзывов на фильмы: строим простые модели
В этом задании вам предлагается начать разбираться с задачей анализа тональности отзывов на примере сентимент-анализа отзывов на фильмы.

Мы будем использовать стандартный датасет из nltk, уже возникавший в одном из примеров в предыдущих курсах. Для того, чтобы импортировать необходимый модуль, напишите:

In [2]:
from nltk.corpus import movie_reviews

Чтобы получить id-шники негативных и позитивных отзывов:

In [3]:
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

Чтобы получить список негативных отзывов:

In [4]:
negfeats = [movie_reviews.words(fileids=[f]) for f in negids]

```
В некоторых пунктах нужно получить ответ - число или строку, которые будет нужно набирать в текстовых файлах и прикреплять в ответах на вопросы. Десятичные дроби записывайте через точку.
```

In [5]:
def print_to_file(value, name):
    print(name, value)
    with open(name, "w") as file:
        file.write(str(value))

1. Создайте список из текстов всех имеющихся отзывов, а также список с классами, которые будет использовать ваш классификатор - 0 для негативных отзывов и 1 для позитивных.

In [6]:
neg_classes = [0] * len(negids)
pos_classes = [1] * len(posids)
classes = neg_classes + pos_classes
negfeats = [movie_reviews.words(fileids=[f]) for f in negids]
posfeats = [movie_reviews.words(fileids=[f]) for f in posids]
reviews = negfeats + posfeats

2. Подсчитайте количество отзывов в выборке.

In [7]:
reviews_cnt = len(reviews)
print_to_file(reviews_cnt, '1.txt')

1.txt 2000


3. Подсчитайте долю класса 1 в выборке.

In [8]:
pos_reviews_cnt = len(posids)
pos_share = pos_reviews_cnt / reviews_cnt
print_to_file(pos_share, '2.txt')

2.txt 0.5


4. Импортируйте CountVectorizer из sklearn.feature_extraction.text. Попробуйте использовать его с настройками по умолчанию для того, чтобы получить признаковое представление каждого текста. Скорее всего, попытка не увенчается успехом. Разберитесь, в чем причина, и добейтесь того, чтобы метод fit_transform у CountVectorizer успешно отрабатывал. Подсчитайте количество признаков в CountVectorizer. Никакой предварительной обработки текста (удаление стоп-слов, нормализация слов) на этом шаге делать не надо, в качестве признаков должны использоваться частоты слов.

In [9]:
vectorizer = CountVectorizer().fit(reviews)
transformed = vectorizer.transform(reviews)

AttributeError: 'StreamBackedCorpusView' object has no attribute 'lower'

In [14]:
shape = transformed.shape[1]
print_to_file(shape, '3.txt')

3.txt 39659


Ошибка
```
'StreamBackedCorpusView' object has no attribute 'lower'
```
говорит о том, что неплохо было бы элементы внутри списка преобразовать в строку, ибо у StreamBackedCorpusView нет аттрибута lower.

In [16]:
type(reviews[0])

str

In [17]:
negfeats = [" ".join(movie_reviews.words(fileids=[f])) for f in negids]
posfeats = [" ".join(movie_reviews.words(fileids=[f])) for f in posids]
reviews = negfeats + posfeats
vectorizer = CountVectorizer()
transformed = vectorizer.fit_transform(reviews)

5. Соберите pipeline из CountVectorizer и LogisticRegression c настройками по-умолчанию и с помощью cross_val_score (также со стандартными настройками) оцените получаемое "из коробки" качество по accuracy.

In [18]:
pipe = Pipeline([('vectorizer', CountVectorizer()), ('log_reg', LogisticRegression(solver='liblinear'))])
acc_scores = cross_val_score(pipe, reviews, classes, scoring = 'accuracy')
print_to_file(acc_scores.mean(), '4.txt')

4.txt 0.8415000000000001


6. Аналогично accuracy, оцените качество по ROC AUC.

In [19]:
roc_auc_scores = cross_val_score(pipe, reviews, classes, scoring = 'roc_auc')
print_to_file(roc_auc_scores.mean(), '5.txt')

5.txt 0.9160600000000001


7. Обучите логистическую регрессию на всей доступной вам выборке и выведите 2! (а не 5 - это ошибка в тексте, грейдер хочет 2 признака) наиболее важных для модели признаков (подумайте, какие именно признаки стоит считать такими). Вам могут пригодиться метод get_feature_names() или поле vocabulary_ у класса CountVectorizer.

In [26]:
model = pipe.fit(reviews, classes)

In [27]:
features = np.array(vectorizer.get_feature_names())
coefs = np.abs(model.named_steps['log_reg'].coef_[0])
coefs_argsort = coefs.argsort()[:-3:-1]
top_5_features = ' '.join(features[coefs_argsort])

In [28]:
print_to_file(top_5_features, '6.txt')

6.txt bad unfortunately
